In [ ]:
import pandas as pd
from pathlib import Path

# Paths
input_path = Path("elections.csv")
output_path = Path("elections_with_perks.csv")

# Perks domain
perks = [
    'Perks.ATimeForGiving', 'Perks.ArcaneCatalyst', 'Perks.AstralNegotiator', 'Perks.Benediction',
    'Perks.BloomingBusiness', 'Perks.Bribe', 'Perks.ChivalrousCarnival', 'Perks.DarkerAuctions',
    'Perks.DoubleMobsHP', 'Perks.DoubleTrouble', 'Perks.EZPZ', 'Perks.ExtraEvent',
    'Perks.ExtraEventFishing_Festival', 'Perks.ExtraEventMining_Fiesta', 'Perks.ExtraEventSpooky_Festival',
    'Perks.ExtraEventSweet_Tooth', 'Perks.FishingFestival', 'Perks.FishingXPBuff', 'Perks.GOATed',
    'Perks.Jerrypocalypse', 'Perks.LongTermInvestment', 'Perks.LuckOfTheSea', 'Perks.Lucky',
    'Perks.MagicXPBoost', 'Perks.Marauder', 'Perks.MiningFiesta', 'Perks.MiningXPBuff', 'Perks.MoarSkillz',
    'Perks.MoltenForge', 'Perks.MythologicalRitual', 'Perks.Pathfinder', 'Perks.PeltPocalypse',
    'Perks.Perkpocalypse', 'Perks.PestEradicator', 'Perks.PetXPBuff', 'Perks.Prospection', 'Perks.QuadTaxes',
    'Perks.SharingIsCaring', 'Perks.ShoppingSpree', 'Perks.SlashedPricing', 'Perks.SlayerXPBuff',
    'Perks.Statspocalypse', 'Perks.StockExchange', 'Perks.SweetBenevolence', 'Perks.TurboMinions',
    'Perks.VolumeTrading'
]

# Load the CSV
if not input_path.exists():
    raise FileNotFoundError(f"Input file not found: {input_path.resolve()}")

df = pd.read_csv(input_path)

# Add missing perk columns with 0
for perk in perks:
    if perk not in df.columns:
        df[perk] = 0

# Save updated CSV
df.to_csv(output_path, index=False)
print(f"Wrote {output_path} with {len(perks)} perk columns added if missing.")


Wrote elections_with_perks.csv with 46 perk columns added if missing.


In [5]:
import pandas as pd
from pathlib import Path

# Files
perks_csv = Path("elections_with_perks.csv")
if not perks_csv.exists():
    raise FileNotFoundError(f"Missing {perks_csv.resolve()} — run the earlier step to create it.")

# Mayor → Perks mapping (provided)
mayor_to_perks = {
  "Aatrox": ["Perks.Pathfinder", "Perks.SlashedPricing", "Perks.SlayerXPBuff"],
  "Barry": ["Perks.ArcaneCatalyst", "Perks.AstralNegotiator", "Perks.MagicXPBoost"],
  "Cole": ["Perks.MiningFiesta", "Perks.MiningXPBuff", "Perks.MoltenForge", "Perks.Prospection"],
  "Derpy": ["Perks.DoubleMobsHP", "Perks.MoarSkillz", "Perks.QuadTaxes", "Perks.TurboMinions"],
  "Diana": ["Perks.Lucky", "Perks.MythologicalRitual", "Perks.PetXPBuff", "Perks.SharingIsCaring"],
  "Diaz": ["Perks.LongTermInvestment", "Perks.ShoppingSpree", "Perks.StockExchange", "Perks.VolumeTrading"],
  "Finnegan": ["Perks.BloomingBusiness", "Perks.GOATed", "Perks.PeltPocalypse", "Perks.PestEradicator"],
  "Foxy": ["Perks.ATimeForGiving", "Perks.ChivalrousCarnival", "Perks.ExtraEventFishing_Festival", "Perks.ExtraEventMining_Fiesta", "Perks.ExtraEventSpooky_Festival", "Perks.ExtraEventSweet_Tooth", "Perks.SweetBenevolence"],
  "Jerry": ["Perks.Jerrypocalypse", "Perks.Perkpocalypse", "Perks.Statspocalypse"],
  "Marina": ["Perks.DoubleTrouble", "Perks.FishingFestival", "Perks.FishingXPBuff", "Perks.LuckOfTheSea"],
  "Paul": ["Perks.Benediction", "Perks.EZPZ", "Perks.Marauder"],
  "Scorpius": ["Perks.Bribe", "Perks.DarkerAuctions"]
}

# Build lookups
# 1) perk -> mayor group label
perk_to_group = {}
for mayor, perks in mayor_to_perks.items():
    for p in perks:
        perk_to_group[p] = mayor

# Load CSV with existing perk columns
df = pd.read_csv(perks_csv)

# Identify all perk columns present (those in the mapping and in df)
perk_columns = [p for p in perk_to_group.keys() if p in df.columns]

# Function to apply weights for a single row
def apply_weights(row):
    # Reset all mapped perk columns to 0 for this row
    for p in perk_columns:
        row[p] = 0.0

    raw_perks = str(row.get("perks", ""))
    if not raw_perks:
        return row

    # Parse perks present in this row and filter to mapped perks
    row_perks = [x.strip() for x in raw_perks.split(",") if x.strip() in perk_to_group]

    # Count how many perks from each mayor's group are present in THIS row
    group_counts = {}
    for p in row_perks:
        g = perk_to_group[p]
        group_counts[g] = group_counts.get(g, 0) + 1

    # Assign weights: 1 / (count of perks in that group in this row)
    for p in row_perks:
        g = perk_to_group[p]
        denom = group_counts.get(g, 0)
        if denom > 0:
            row[p] = 1.0 / denom

    return row

# Apply to every row
df = df.apply(apply_weights, axis=1)

# Save back
df.to_csv(perks_csv, index=False)
print(f"Updated weighted perks written to {perks_csv}")


Updated weighted perks written to elections_with_perks.csv


In [6]:
import re
import pandas as pd
from pathlib import Path

# Inputs
lua_path = Path("mayor_data.txt")
perks_csv = Path("elections_with_perks.csv")
if not lua_path.exists():
    raise FileNotFoundError(f"Missing {lua_path.resolve()}")
if not perks_csv.exists():
    raise FileNotFoundError(f"Missing {perks_csv.resolve()}")

# Mayor → Perks mapping (same as earlier cell)
mayor_to_perks = {
  "Aatrox": ["Perks.Pathfinder", "Perks.SlashedPricing", "Perks.SlayerXPBuff"],
  "Barry": ["Perks.ArcaneCatalyst", "Perks.AstralNegotiator", "Perks.MagicXPBoost"],
  "Cole": ["Perks.MiningFiesta", "Perks.MiningXPBuff", "Perks.MoltenForge", "Perks.Prospection"],
  "Derpy": ["Perks.DoubleMobsHP", "Perks.MoarSkillz", "Perks.QuadTaxes", "Perks.TurboMinions"],
  "Diana": ["Perks.Lucky", "Perks.MythologicalRitual", "Perks.PetXPBuff", "Perks.SharingIsCaring"],
  "Diaz": ["Perks.LongTermInvestment", "Perks.ShoppingSpree", "Perks.StockExchange", "Perks.VolumeTrading"],
  "Finnegan": ["Perks.BloomingBusiness", "Perks.GOATed", "Perks.PeltPocalypse", "Perks.PestEradicator"],
  "Foxy": ["Perks.ATimeForGiving", "Perks.ChivalrousCarnival", "Perks.ExtraEventFishing_Festival", "Perks.ExtraEventMining_Fiesta", "Perks.ExtraEventSpooky_Festival", "Perks.SweetBenevolence"],
  "Jerry": ["Perks.Jerrypocalypse", "Perks.Perkpocalypse", "Perks.Statspocalypse"],
  "Marina": ["Perks.DoubleTrouble", "Perks.FishingFestival", "Perks.FishingXPBuff", "Perks.LuckOfTheSea"],
  "Paul": ["Perks.Benediction", "Perks.EZPZ", "Perks.Marauder"],
  "Scorpius": ["Perks.Bribe", "Perks.DarkerAuctions"]
}

# Build helpers
perk_to_group = {}
group_to_perks = {}
for mayor, perks in mayor_to_perks.items():
    group_to_perks[mayor] = set(perks)
    for p in perks:
        perk_to_group[p] = mayor

# Parse Lua file to extract minister perks per election
election_to_minister_perks = {}
current_election = None
in_minister = False
brace_depth = 0
perk_regex = re.compile(r"Perks\.[A-Za-z_]+")
num_regex = re.compile(r"^\s*\[(\d+)\]\s*=")

with lua_path.open("r", encoding="utf-8") as f:
    for line in f:
        # detect election start
        m = num_regex.search(line)
        if m:
            current_election = int(m.group(1))
            in_minister = False
            brace_depth = 0
            continue

        if current_election is None:
            continue

        # enter minister block
        if not in_minister and "minister" in line and "{" in line:
            in_minister = True
            brace_depth = line.count("{") - line.count("}")
            # capture any perks on same line
            for perk in perk_regex.findall(line):
                election_to_minister_perks.setdefault(current_election, set()).add(perk)
            continue

        if in_minister:
            # update depth and collect perks
            brace_depth += line.count("{")
            for perk in perk_regex.findall(line):
                election_to_minister_perks.setdefault(current_election, set()).add(perk)
            brace_depth -= line.count("}")
            if brace_depth <= 0:
                in_minister = False
                # finished minister block

# Load CSV and ensure needed columns exist
df = pd.read_csv(perks_csv)
all_mapped_perks = set(perk_to_group.keys())
for p in all_mapped_perks:
    if p not in df.columns:
        df[p] = 0.0

# Apply minister perk overrides per row
def apply_minister_overrides(row):
    try:
        election_num = int(row.get("election_number"))
    except (TypeError, ValueError):
        return row
    minister_perks = election_to_minister_perks.get(election_num)
    if not minister_perks:
        return row

    for perk in minister_perks:
        if perk not in perk_to_group:
            continue
        group = perk_to_group[perk]
        # set target perk to 1
        if perk in row.index:
            row[perk] = 1.0
        # set all other perks in the same group to 0
        for other in group_to_perks[group]:
            if other == perk:
                continue
            if other in row.index:
                row[other] = 0.0
    return row

# Update rows
df = df.apply(apply_minister_overrides, axis=1)

df.to_csv(perks_csv, index=False)
print("Applied minister overrides to", perks_csv)


Applied minister overrides to elections_with_perks.csv


In [1]:
import json
import re
import pandas as pd
from pathlib import Path

# Inputs
json_path = Path("election_with_ministers_year400+.json")
csv_path = Path("elections_with_perks_updated.csv")
if not json_path.exists():
    raise FileNotFoundError(f"Missing {json_path.resolve()}")
if not csv_path.exists():
    raise FileNotFoundError(f"Missing {csv_path.resolve()} — ensure the CSV exists before running this cell.")

# Load CSV
df = pd.read_csv(csv_path)

# Helper: normalize display names for robust matching
_normalize = lambda s: re.sub(r"[^A-Za-z0-9]", "", str(s)).lower()

# Explicit mapping from JSON display names -> standardized Perks.* column names
name_to_col = {
    # Events / Foxy
    "A Time for Giving": "Perks.ATimeForGiving",
    "Sweet Benevolence": "Perks.SweetBenevolence",
    "Extra Event": "Perks.ExtraEvent",
    "Extra Event: Fishing Festival": "Perks.ExtraEventFishing_Festival",
    "Extra Event: Mining Fiesta": "Perks.ExtraEventMining_Fiesta",
    "Extra Event: Spooky Festival": "Perks.ExtraEventSpooky_Festival",
    "Extra Event: Sweet Tooth": "Perks.ExtraEventSweet_Tooth",

    # Diaz / Economist
    "Stock Exchange": "Perks.StockExchange",
    "Long Term Investment": "Perks.LongTermInvestment",
    "Shopping Spree": "Perks.ShoppingSpree",
    "Volume Trading": "Perks.VolumeTrading",

    # Diana / Pets
    "Mythological Ritual": "Perks.MythologicalRitual",
    "Sharing is Caring": "Perks.SharingIsCaring",
    "Lucky": "Perks.Lucky",
    "Pet XP Buff": "Perks.PetXPBuff",

    # Marina / Fishing
    "Fishing Festival": "Perks.FishingFestival",
    "Fishing XP Buff": "Perks.FishingXPBuff",
    "Luck of the Sea": "Perks.LuckOfTheSea",
    "Luck of the Sea 2.0": "Perks.LuckOfTheSea",
    "Double Trouble": "Perks.DoubleTrouble",

    # Paul / Dungeons
    "Benediction": "Perks.Benediction",
    "EZPZ": "Perks.EZPZ",
    "Marauder": "Perks.Marauder",

    # Aatrox / Slayer
    "SLASHED Pricing": "Perks.SlashedPricing",
    "Slashed Pricing": "Perks.SlashedPricing",
    "Slayer XP Buff": "Perks.SlayerXPBuff",
    "Pathfinder": "Perks.Pathfinder",

    # Jerry
    "Jerrypocalypse": "Perks.Jerrypocalypse",
    "Perkpocalypse": "Perks.Perkpocalypse",
    "Statspocalypse": "Perks.Statspocalypse",

    # Scorpius
    "Bribe": "Perks.Bribe",
    "Darker Auctions": "Perks.DarkerAuctions",

    # Barry / Wizardy
    "Arcane Catalyst": "Perks.ArcaneCatalyst",
    "Astral Negotiator": "Perks.AstralNegotiator",
    "Magic XP Boost": "Perks.MagicXPBoost",

    # Cole / Mining
    "Mining Fiesta": "Perks.MiningFiesta",
    "Mining XP Buff": "Perks.MiningXPBuff",
    "Molten Forge": "Perks.MoltenForge",
    "Prospection": "Perks.Prospection",

    # Derpy
    "Double Mobs HP": "Perks.DoubleMobsHP",
    "Moar Skillz": "Perks.MoarSkillz",
    "Quad Taxes": "Perks.QuadTaxes",
    "Turbo Minions": "Perks.TurboMinions",

    # Finnegan / Farming
    "Blooming Business": "Perks.BloomingBusiness",
    "GOATed": "Perks.GOATed",
    "Pelt-pocalypse": "Perks.PeltPocalypse",
    "Pelt Pocalypse": "Perks.PeltPocalypse",
    "Pest Eradicator": "Perks.PestEradicator",

    # Misc / festivals
    "Chivalrous Carnival": "Perks.ChivalrousCarnival",
}

# Build a normalized lookup as fallback
norm_lookup = {_normalize(k): v for k, v in name_to_col.items()}

# Load JSON entries
with json_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

# Ensure election_number column exists and is comparable
if "election_number" not in df.columns:
    raise KeyError("CSV is missing 'election_number' column to match JSON 'year'.")

updates = 0
missing_map = set()
missing_years = set()

for entry in data:
    year = entry.get("year")
    if year is None:
        continue
    # Find matching rows in CSV
    mask = df["election_number"].astype("Int64") == int(year)
    if not mask.any():
        missing_years.add(int(year))
        continue

    # Iterate candidates' perks, apply only minister perks
    for cand in entry.get("candidates", []):
        for perk in cand.get("perks", []):
            if not perk.get("minister", False):
                continue
            display_name = perk.get("name", "")
            col = name_to_col.get(display_name)
            if col is None:
                col = norm_lookup.get(_normalize(display_name))
            if col is None:
                missing_map.add(display_name)
                continue
            # Ensure column exists
            if col not in df.columns:
                df[col] = 0
            # Set to 1 for all rows matching this election year
            df.loc[mask, col] = 1
            updates += int(mask.sum())

# Save back
df.to_csv(csv_path, index=False)
print(f"Updated {csv_path} — applied {updates} minister perk flags.")
if missing_map:
    print("Unmapped perk names (please extend mapping):", sorted(missing_map))
if missing_years:
    print("JSON years not found in CSV election_number:", sorted(missing_years))



Updated elections_with_perks_updated.csv — applied 205 minister perk flags.


In [4]:
import pandas as pd
from pathlib import Path

# Input and output paths
input_path = Path("elections_with_perks.csv")
output_path = Path("elections_with_perks_filtered_no_minister.csv")

# Check if input file exists
if not input_path.exists():
    raise FileNotFoundError(f"Input file not found: {input_path.resolve()}")

# Load the CSV
df = pd.read_csv(input_path)

# Display original data info
print(f"Original dataset shape: {df.shape}")
print(f"Original number of rows: {len(df)}")

# Check what candidate columns exist
candidate_columns = [col for col in df.columns if 'candidate' in col.lower()]
print(f"Available candidate columns: {candidate_columns}")

# Candidates to remove
candidates_to_remove = ['Jerry', 'Scorpius', 'Derpy']

# Create a mask to identify rows to keep (rows that don't have any of the specified candidates)
rows_to_keep = pd.Series([True] * len(df))

# Check each candidate column for the specified candidates
for col in candidate_columns:
    if col in df.columns:
        # Create mask for rows that don't contain any of the specified candidates in this column
        mask = ~df[col].isin(candidates_to_remove)
        rows_to_keep = rows_to_keep & mask

# Apply the filter
filtered_df = df[rows_to_keep].copy()

# Display filtered data info
print(f"Filtered dataset shape: {filtered_df.shape}")
print(f"Filtered number of rows: {len(filtered_df)}")
print(f"Removed {len(df) - len(filtered_df)} rows containing Jerry, Scorpius, or Derpy")

# Save the filtered data to a new CSV
filtered_df.to_csv(output_path, index=False)
print(f"Filtered data saved to: {output_path}")

# Show some statistics about what was removed
print("\nRemoved rows breakdown:")
for candidate in candidates_to_remove:
    count = 0
    for col in candidate_columns:
        if col in df.columns:
            count += (df[col] == candidate).sum()
    print(f"  Rows with {candidate}: {count}")


Original dataset shape: (265, 52)
Original number of rows: 265
Available candidate columns: ['candidates']
Filtered dataset shape: (265, 52)
Filtered number of rows: 265
Removed 0 rows containing Jerry, Scorpius, or Derpy
Filtered data saved to: elections_with_perks_filtered_no_minister.csv

Removed rows breakdown:
  Rows with Jerry: 0
  Rows with Scorpius: 0
  Rows with Derpy: 0


In [7]:
import pandas as pd
from pathlib import Path

# Input and output paths
input_path = Path("elections_with_perks_filtered_no_minister.csv")
output_path = Path("elections_with_perks_no_special_no_minister.csv")

# Check if input file exists
if not input_path.exists():
    raise FileNotFoundError(f"Input file not found: {input_path.resolve()}")

# Load the CSV
df = pd.read_csv(input_path)

# Mayor to perks mapping (from the dictionary)
mayor_to_perks = {
  "Aatrox": ["Perks.Pathfinder", "Perks.SlashedPricing", "Perks.SlayerXPBuff"],
  "Barry": ["Perks.ArcaneCatalyst", "Perks.AstralNegotiator", "Perks.MagicXPBoost"],
  "Cole": ["Perks.MiningFiesta", "Perks.MiningXPBuff", "Perks.MoltenForge", "Perks.Prospection"],
  "Derpy": ["Perks.DoubleMobsHP", "Perks.MoarSkillz", "Perks.QuadTaxes", "Perks.TurboMinions"],
  "Diana": ["Perks.Lucky", "Perks.MythologicalRitual", "Perks.PetXPBuff", "Perks.SharingIsCaring"],
  "Diaz": ["Perks.LongTermInvestment", "Perks.ShoppingSpree", "Perks.StockExchange", "Perks.VolumeTrading"],
  "Finnegan": ["Perks.BloomingBusiness", "Perks.GOATed", "Perks.PeltPocalypse", "Perks.PestEradicator"],
  "Foxy": ["Perks.ATimeForGiving", "Perks.ChivalrousCarnival", "Perks.ExtraEventFishing_Festival", "Perks.ExtraEventMining_Fiesta", "Perks.ExtraEventSpooky_Festival", "Perks.SweetBenevolence"],
  "Jerry": ["Perks.Jerrypocalypse", "Perks.Perkpocalypse", "Perks.Statspocalypse"],
  "Marina": ["Perks.DoubleTrouble", "Perks.FishingFestival", "Perks.FishingXPBuff", "Perks.LuckOfTheSea"],
  "Paul": ["Perks.Benediction", "Perks.EZPZ", "Perks.Marauder"],
  "Scorpius": ["Perks.Bribe", "Perks.DarkerAuctions"]
}

# Get perks to remove (Jerry, Scorpius, and Derpy)
mayors_to_remove = ['Jerry', 'Scorpius', 'Derpy']
perks_to_remove = []

for mayor in mayors_to_remove:
    if mayor in mayor_to_perks:
        perks_to_remove.extend(mayor_to_perks[mayor])

print(f"Original dataset shape: {df.shape}")
print(f"Perks to remove: {perks_to_remove}")

# Check which perk columns actually exist in the dataset
existing_perks_to_remove = [perk for perk in perks_to_remove if perk in df.columns]
missing_perks = [perk for perk in perks_to_remove if perk not in df.columns]

print(f"Existing perk columns to remove: {existing_perks_to_remove}")
if missing_perks:
    print(f"Perk columns not found in dataset: {missing_perks}")

# Remove the perk columns
df_filtered = df.drop(columns=existing_perks_to_remove, errors='ignore')

print(f"Filtered dataset shape: {df_filtered.shape}")
print(f"Removed {len(existing_perks_to_remove)} perk columns")

# Save the filtered data to a new CSV
df_filtered.to_csv(output_path, index=False)
print(f"Filtered data saved to: {output_path}")

# Show remaining perk columns
remaining_perk_columns = [col for col in df_filtered.columns if col.startswith('Perks.')]
print(f"Remaining perk columns ({len(remaining_perk_columns)}): {remaining_perk_columns}")


Original dataset shape: (265, 52)
Perks to remove: ['Perks.Jerrypocalypse', 'Perks.Perkpocalypse', 'Perks.Statspocalypse', 'Perks.Bribe', 'Perks.DarkerAuctions', 'Perks.DoubleMobsHP', 'Perks.MoarSkillz', 'Perks.QuadTaxes', 'Perks.TurboMinions']
Existing perk columns to remove: ['Perks.Jerrypocalypse', 'Perks.Perkpocalypse', 'Perks.Statspocalypse', 'Perks.Bribe', 'Perks.DarkerAuctions', 'Perks.DoubleMobsHP', 'Perks.MoarSkillz', 'Perks.QuadTaxes', 'Perks.TurboMinions']
Filtered dataset shape: (265, 43)
Removed 9 perk columns
Filtered data saved to: elections_with_perks_no_special_no_minister.csv
Remaining perk columns (37): ['Perks.ATimeForGiving', 'Perks.ArcaneCatalyst', 'Perks.AstralNegotiator', 'Perks.Benediction', 'Perks.BloomingBusiness', 'Perks.ChivalrousCarnival', 'Perks.DoubleTrouble', 'Perks.EZPZ', 'Perks.ExtraEvent', 'Perks.ExtraEventFishing_Festival', 'Perks.ExtraEventMining_Fiesta', 'Perks.ExtraEventSpooky_Festival', 'Perks.ExtraEventSweet_Tooth', 'Perks.FishingFestival', 'Pe